###### https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/

In [1]:
import re
import tweepy
import config
from tweepy import OAuthHandler
from textblob import TextBlob

In [2]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = config.consumer_key
        consumer_secret = config.consumer_secret
        access_token = config.access_token
        access_token_secret = config.access_token_secret
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 30):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search_tweets(q = query, lang = "en" , count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.Error as e:
            # print error (if any)
            print("Error : " + str(e))

In [7]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'Covid19', count = 10)
    print(tweets.text)
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
  
if __name__ == "__main__":
    # calling main function
    main()
    #print(tweets.count)

[{'text': 'RT @NReveillee: Pros/Cons: Regeneron vs Pfizer vaccine for Covid19\n\n➡️Regeneron Effective ⬇️symptoms of Covid19 in 62% of wider population…', 'sentiment': 'positive'}, {'text': 'RT @JohnBasham: REPORT: #Houston #Texas Doctor Claims He Has Succesfully Used Ivermectin To Treat Thousands Of #COVID19 Patients Without Si…', 'sentiment': 'neutral'}, {'text': 'RT @KanchanGupta: Tale of 2 States:\n#India fresh #COVID19 cases: 37,593\n\n#Kerala\n31,445 fresh cases\n215 deaths\n19.03 TPR\n\n#UttarPradesh\n22…', 'sentiment': 'positive'}, {'text': 'RT @sarahvmac: Visceral radio TWT young woman talking about ambos coming to get her husband last night. Bravery of them walking into house…', 'sentiment': 'positive'}, {'text': 'RT @PalimenoForGAD1: @GovRonDeSantis Now how about you protect the lives of your constituents from #COVID19', 'sentiment': 'neutral'}, {'text': 'Mask, social distancing and vaccination are ineffective against airborne virus that #Covid19 is. No matter how many… http